In [2]:
from google.colab import drive
drive.mount("/content/gdrive")
%cd '/content/gdrive/MyDrive/LDS6_S7N_TranHoangBach/BT_Plus/Chapter_7'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/LDS6_S7N_TranHoangBach/BT_Plus/Chapter_7
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/LDS6_S7N_TranHoangBach/BT_Plus/Chapter_7


In [ ]:
!pip install pandas-profiling==2.7.1

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import pandas_profiling as pp
%matplotlib inline

In [5]:
df = pd.read_csv('Absenteeism_at_work.csv', sep=';')
df

,ID,Reason for absence,Month of absence,Day of the week,Seasons,Transportation expense,Distance from Residence to Work,Service time,Age,Work load Average per day,Hit target,Disciplinary failure,Education,Son,Social drinker,Social smoker,Pet,Weight,Height,Body mass index,Absenteeism time in hours
0,11,26,7,3,1,289,36,13,33,239.554,97,0,1,2,1,0,1,90,172,30,4
1,36,0,7,3,1,118,13,18,50,239.554,97,1,1,1,1,0,0,98,178,31,0
2,3,23,7,4,1,179,51,18,38,239.554,97,0,1,0,1,0,0,89,170,31,2
3,7,7,7,5,1,279,5,14,39,239.554,97,0,1,2,1,1,0,68,168,24,4
4,11,23,7,5,1,289,36,13,33,239.554,97,0,1,2,1,0,1,90,172,30,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
735,11,14,7,3,1,289,36,13,33,264.604,93,0,1,2,1,0,1,90,172,30,8
736,1,11,7,3,1,235,11,14,37,264.604,93,0,3,1,0,0,1,88,172,29,4
737,4,0,0,3,1,118,14,13,40,271.219,95,0,1,1,1,0,8,98,170,34,0
738,8,0,0,4,2,231,35,14,39,271.219,95,0,1,2,1,0,2,100,170,35,0


In [6]:
profile = pp.ProfileReport(df)
profile

Output hidden; open in https://colab.research.google.com to view.

Phân tích dữ liệu:
- Data có 740 mẫu, 21 features. tuy nhiên có một vài dòng bị duplicate.
- Dữ liệu không có missing values.
- Dữ liệu đã được số hoá (converted to real numbers), tuy nhiên cần phân biệt rõ biến nào là categorical variables, biến nào là numerical variables.
- Target là **'Absenteeism time in hours'**, có 2 cách để thực hiện bài toán này, đầu tiên thử **Regression**.
- Vì số lượng categorical variables khá nhiều, nên nếu sử dụng các model như Linear Regression, Polynorminal, SVM thì cần convert sang one-hot encoding. Lúc đó số lượng features sẽ rất nhiều so với sample nên dễ bị Overfiting.
- Nên sử dụng model thuộc nhóm Tree như Random Forest, Boosting, khi đó data không cần phải convert sang one-hot encoding. 
- Đồng thời các numerical variables không cần phải Scaling vì thuật toán Tree không sử dụng các phương pháp tính khoảng cách hay Gradient Descent.
- Để Random Forest, Boosting không bị overfiting, cần tuning model bằng ccp_alphas.

In [7]:
categoric = ['Reason for absence', 'Month of absence', 'Day of the week', 'Seasons', 'Disciplinary failure', 'Education', 'Social drinker', 'Social smoker']
numeric = ['Transportation expense', 'Distance from Residence to Work', 'Service time', 'Age', 'Work load Average per day', 'Hit target', 'Son', 'Pet', 'Weight', 'Height', 'Body mass index']
target = 'Absenteeism time in hours'

In [8]:
df = df.drop_duplicates()
df.shape

(706, 21)

- Xem xét thêm các biến có đa cộng tuyến với nhau hay không? 
- Nếu có thì loại bỏ features bị đa cộng tuyến đi.
- Sử dụng chỉ số VIF (Variance Inflation Factor), nếu VIF > 5 thì biến bị đa cộng tuyến

In [9]:
# Xem vấn đề có đa cộng tuyến trong model hay không?
from statsmodels.tools.tools import add_constant
X_full = df.drop(columns=target)
X_full = add_constant(X_full)

from statsmodels.stats.outliers_influence import variance_inflation_factor
print('Variance Inflation Factors')
for i in range(X_full.shape[1]):
    print('{:35}'.format(X_full.columns[i]), '{:.3f}'.format(variance_inflation_factor(X_full.values,i)))

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Variance Inflation Factors
const                               27845.506
ID                                  2.203
Reason for absence                  1.546
Month of absence                    1.818
Day of the week                     1.090
Seasons                             1.428
Transportation expense              1.840
Distance from Residence to Work     2.525
Service time                        3.805
Age                                 2.855
Work load Average per day           1.251
Hit target                          1.468
Disciplinary failure                1.544
Education                           1.790
Son                                 1.383
Social drinker                      2.683
Social smoker                       1.328
Pet                                 1.851
Weight                              177.022
Height                              33.358
Body mass index                     162.357


- Các biến Weight, Height, BMI bị đa cộng tuyến lẫn nhau.
- Loại bỏ biến Weight, và xem lại kết quả

In [10]:
# Xem vấn đề có đa cộng tuyến trong model hay không?
from statsmodels.tools.tools import add_constant
df = df.drop(columns=['Weight'])
X_full = df.drop(columns=[target])
X_full = add_constant(X_full)

from statsmodels.stats.outliers_influence import variance_inflation_factor
print('Variance Inflation Factors')
for i in range(X_full.shape[1]):
    print('{:35}'.format(X_full.columns[i]), '{:.3f}'.format(variance_inflation_factor(X_full.values,i)))

Variance Inflation Factors
const                               2672.633
ID                                  2.141
Reason for absence                  1.546
Month of absence                    1.794
Day of the week                     1.086
Seasons                             1.401
Transportation expense              1.830
Distance from Residence to Work     2.433
Service time                        3.458
Age                                 2.714
Work load Average per day           1.243
Hit target                          1.459
Disciplinary failure                1.541
Education                           1.741
Son                                 1.379
Social drinker                      2.645
Social smoker                       1.266
Pet                                 1.677
Height                              1.710
Body mass index                     2.142


Đã hết hiện tượng đa cộng tuyến

In [11]:
numeric.remove('Weight')

In [12]:
# Viết function pre-processing
def pre_processing(data, numeric_col=numeric, categoric_col=categoric, target_col=target, one_hot=False):
    df_num = data[numeric_col]
    # One-hot
    if one_hot:
        df_cate = pd.get_dummies(data[categoric_col], columns=categoric_col, drop_first=True)
    else:
        df_cate = data[categoric_col]

    df_target = data[[target_col]]
    df = pd.concat([df_num, df_cate, df_target], axis=1)

    # Split train-test
    from sklearn.model_selection import train_test_split
    X = df.drop(columns=[target_col])
    y = df[target_col]
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
    
    # Sử dụng Robust Scaler
    from sklearn.preprocessing import RobustScaler
    scaler = RobustScaler()
    X_train[numeric] = scaler.fit_transform(X_train[numeric])
    X_test[numeric] = scaler.transform(X_test[numeric])

    return X_train, X_test, y_train, y_test

In [ ]:
data_set_1 = pre_processing(df, one_hot=False)
data_set_2 = pre_processing(df, one_hot=True)

In [14]:
def result_model_regr_cv(list_model, data_set, cv_k=5, show_table=False):
    import warnings
    warnings.filterwarnings("ignore")
    from sklearn.model_selection import cross_validate, KFold
    from sklearn.metrics import mean_squared_error

    X_train, X_test, y_train, y_test = data_set

    kfold = KFold(n_splits=cv_k, shuffle=True, random_state=42)
    result = []
    for name, model in list_model:
        cv_result = cross_validate(model, X_train, y_train, cv=kfold, return_train_score=True, scoring=['neg_mean_squared_error', 'r2'])
        model.fit(X_train, y_train)
        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)

        result.append([name, 
                       np.abs(cv_result['train_neg_mean_squared_error'].mean()), 
                       np.abs(cv_result['test_neg_mean_squared_error'].mean()), 
                       cv_result['train_r2'].mean(),
                       cv_result['test_r2'].mean(),
                       cv_result['fit_time'].mean(),
                       cv_result['score_time'].mean(),
                       mean_squared_error(y_train, y_pred_train),
                       mean_squared_error(y_test, y_pred_test),
                       model.score(X_train, y_train),
                       model.score(X_test, y_test),
        ])

    result = pd.DataFrame(result, columns=['model', 'train_cv_mse', 'test_cv_mse',  
                                           'train_cv_r2', 'test_cv_r2', 'fit_time', 'score_time',
                                           'train_mse', 'test_mse', 
                                           'train_r2', 'test_r2',
                                           ])
    result.sort_values('test_cv_mse', ascending=True, inplace=True)
    if show_table:
        display(result)
    return result

In [15]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from xgboost import XGBRegressor

list_model = [
              ('Linear Regression', LinearRegression()),
            #   ('KNN (3)', KNeighborsRegressor(3)),
              ('KNN (5)', KNeighborsRegressor(5)),
            #   ('KNN (8)', KNeighborsRegressor(8)),
              ('SVR - linear', SVR(kernel='linear')),
              ('SVR - poly 2', SVR(kernel='poly', degree=2)),
            #   ('SVR - poly 3', SVR(kernel='poly', degree=3)),
              ('SVR - rbf', SVR(kernel='rbf')),
              ('Random Forest', RandomForestRegressor(n_estimators=250)),
            #   ('AdaBoost', AdaBoostRegressor(base_estimator=RandomForestRegressor(random_state=42), random_state=42)),
            #   ('XGBoost', XGBRegressor()),
]

In [16]:
result_model_regr_cv(list_model, data_set_1)

,model,train_cv_mse,test_cv_mse,train_cv_r2,test_cv_r2,fit_time,score_time,train_mse,test_mse,train_r2,test_r2
1,KNN (5),111.179603,157.386932,0.389880,0.061609,0.006632,0.008996,115.668204,168.581469,0.361493,0.140330
0,Linear Regression,151.261217,164.093174,0.164876,0.046699,0.003955,0.002698,152.530869,175.929773,0.158005,0.102858
4,SVR - rbf,173.573878,175.376824,0.041864,0.054755,0.025310,0.005614,173.778399,187.263756,0.040716,0.045061
2,SVR - linear,174.665441,176.809042,0.036008,0.044806,0.130921,0.004697,174.428517,191.007151,0.037127,0.025971
3,SVR - poly 2,176.898556,177.426972,0.023417,0.033842,0.020220,0.004290,176.666684,192.415357,0.024772,0.018790
5,Random Forest,33.426649,186.980844,0.815804,-0.280381,1.177263,0.028502,32.403756,165.639437,0.821126,0.155332


In [17]:
result_model_regr_cv(list_model, data_set_2)

,model,train_cv_mse,test_cv_mse,train_cv_r2,test_cv_r2,fit_time,score_time,train_mse,test_mse,train_r2,test_r2
2,SVR - linear,165.170640,1.675480e+02,0.089323,1.124650e-01,0.060832,0.004944,165.379435,1.837976e+02,0.087079,6.273589e-02
4,SVR - rbf,178.916708,1.836681e+02,0.013135,1.628549e-05,0.031512,0.008228,178.003216,1.920481e+02,0.017394,2.066322e-02
3,SVR - poly 2,179.799843,1.860197e+02,0.007915,-2.337664e-02,0.024555,0.005159,178.991433,1.982588e+02,0.011939,-1.100775e-02
5,Random Forest,30.178658,1.882823e+02,0.833368,3.022949e-02,1.080338,0.020201,32.301970,2.040492e+02,0.821688,-4.053558e-02
1,KNN (5),131.335844,1.918998e+02,0.276365,-8.367631e-02,0.004328,0.008640,132.700416,1.684038e+02,0.267473,1.412356e-01
0,Linear Regression,124.894818,2.009636e+26,0.315867,-9.634477e+23,0.008100,0.002695,128.080678,3.366997e+26,0.292974,-1.716978e+24


- Model Regression dường như không hiệu quả.
- Thử convert bài toán Regression thành Classification. 

Phân tích bài toán trong thực tế:
- Trong thực tế thì người ta ít khi chia ra và quản lý trên số giờ nghỉ làm, mà thường sẽ chia ra theo buổi, 4h cho 1 buổi (1/2 ngày lương).
- Nên ta thử chia target 'Absenteeism time in hours' thành 4 nhóm: **<= 4h** (nửa ngày), **4-8h** (1/2 - 1 ngày), **8-16h** (1-2 ngày), >16h (nhiều hơn 2 ngày).

In [18]:
labels = ['<=4', '4-8', '8-16', '>16']
bins = [-1, 4, 8, 16, np.inf]
df_clf = df.copy()
df_clf[target] = pd.cut(df_clf[target], bins=bins, labels=labels)
df_clf.head()

,ID,Reason for absence,Month of absence,Day of the week,Seasons,Transportation expense,Distance from Residence to Work,Service time,Age,Work load Average per day,Hit target,Disciplinary failure,Education,Son,Social drinker,Social smoker,Pet,Height,Body mass index,Absenteeism time in hours
0,11,26,7,3,1,289,36,13,33,239.554,97,0,1,2,1,0,1,172,30,<=4
1,36,0,7,3,1,118,13,18,50,239.554,97,1,1,1,1,0,0,178,31,<=4
2,3,23,7,4,1,179,51,18,38,239.554,97,0,1,0,1,0,0,170,31,<=4
3,7,7,7,5,1,279,5,14,39,239.554,97,0,1,2,1,1,0,168,24,<=4
4,11,23,7,5,1,289,36,13,33,239.554,97,0,1,2,1,0,1,172,30,<=4


In [19]:
data_set_1_clf = pre_processing(df_clf, one_hot=False)
data_set_2_clf = pre_processing(df_clf, one_hot=True)

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier

list_model_clf = [
              ('Logistic Regression', LogisticRegression()),
            #   ('KNN (3)', KNeighborsClassifier(3)),
              ('KNN (5)', KNeighborsClassifier(5)),
            #   ('KNN (8)', KNeighborsClassifier(8)),
              ('SVC - linear', SVC(kernel='linear')),
              ('SVC - poly 2', SVC(kernel='poly', degree=2)),
            #   ('SVC - poly 3', SVC(kernel='poly', degree=3)),
              ('SVC - rbf', SVC(kernel='rbf')),
              ('Random Forest', RandomForestClassifier(n_estimators=250)),
            #   ('AdaBoost', AdaBoostClassifier(base_estimator=RandomForestClassifier(random_state=42), random_state=42)),
            #   ('XGBoost', XGBClassifier()),
]

In [21]:
def result_model_clf_cv(list_model, data_set, cv_k=5, show_table=False):
    import warnings
    warnings.filterwarnings("ignore")
    from sklearn.model_selection import cross_validate, KFold
    from sklearn.metrics import accuracy_score, f1_score

    X_train, X_test, y_train, y_test = data_set

    kfold = KFold(n_splits=cv_k, shuffle=True, random_state=42)
    result = []
    for name, model in list_model:
        cv_result = cross_validate(model, X_train, y_train, cv=kfold, return_train_score=True, scoring=['accuracy', 'f1_micro'])
        model.fit(X_train, y_train)
        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)

        result.append([name, 
                       cv_result['train_accuracy'].mean(),
                       cv_result['test_accuracy'].mean(),
                       cv_result['train_f1_micro'].mean(),
                       cv_result['test_f1_micro'].mean(),
                       cv_result['fit_time'].mean(),
                       cv_result['score_time'].mean(),
                       accuracy_score(y_train, y_pred_train),
                       accuracy_score(y_test, y_pred_test),
                    #    f1_score(y_train, y_pred_train, average='micro'),
                    #    f1_score(y_test, y_pred_test, average='micro'),
        ])

    result = pd.DataFrame(result, columns=['model', 'train_cv_accuracy', 'test_cv_accuracy',  
                                           'train_cv_f1', 'test_cv_f1', 'fit_time', 'score_time',
                                           'train_accuracy', 'test_accuracy', 
                                        #    'train_f1', 'test_f1',
                                           ])
    result.sort_values('test_cv_f1', ascending=False, inplace=True)
    if show_table:
        display(result)
    return result

In [22]:
result_model_clf_cv(list_model_clf, data_set_1_clf)

,model,train_cv_accuracy,test_cv_accuracy,train_cv_f1,test_cv_f1,fit_time,score_time,train_accuracy,test_accuracy
5,Random Forest,0.991022,0.697592,0.991022,0.697592,0.452553,0.027175,0.988658,0.706215
2,SVC - linear,0.719282,0.689991,0.719282,0.689991,0.075744,0.003791,0.722117,0.700565
1,KNN (5),0.763706,0.665409,0.763706,0.665409,0.002772,0.007759,0.775047,0.666667
0,Logistic Regression,0.706517,0.661635,0.706517,0.661635,0.047969,0.003341,0.703214,0.672316
4,SVC - rbf,0.667300,0.644582,0.667300,0.644582,0.013738,0.004762,0.659735,0.627119
3,SVC - poly 2,0.601136,0.601168,0.601136,0.601168,0.012773,0.003952,0.601134,0.621469


In [23]:
result_model_clf_cv(list_model_clf, data_set_2_clf)

,model,train_cv_accuracy,test_cv_accuracy,train_cv_f1,test_cv_f1,fit_time,score_time,train_accuracy,test_accuracy
0,Logistic Regression,0.805773,0.710889,0.805773,0.710889,0.058710,0.003551,0.801512,0.779661
2,SVC - linear,0.807187,0.707080,0.807187,0.707080,0.032604,0.005477,0.786389,0.745763
5,Random Forest,0.991022,0.703235,0.991022,0.703235,0.472904,0.027450,0.988658,0.723164
4,SVC - rbf,0.770795,0.665445,0.770795,0.665445,0.029965,0.007555,0.782609,0.711864
3,SVC - poly 2,0.732510,0.644672,0.732510,0.644672,0.022401,0.006079,0.735350,0.677966
1,KNN (5),0.730149,0.618131,0.730149,0.618131,0.003955,0.010865,0.733459,0.700565


- Model Classification cho ra kết quả khá hứa hẹn khi mà ít có hiện hượng overfiting hay underfiting so với model Regression.
- Test cv accuracy các model tốt nhất (Logistic Regression, Random Forest, SVC-linear) khoảng 70%.
- Chỉ có model Random Forest chị Overfiting. Thử AdaBoost, XGBoost xem có cải thiện được hơn Random Forest không.
- Đồng thời tunning các models để xem kết quả như thế nào.

In [24]:
model_tunning_list = [
              ('SVC - linear', SVC(kernel='linear', random_state=42)),
              ('SVC - poly', SVC(kernel='poly', degree=2, random_state=42)),
              ('SVC - rbf', SVC(kernel='rbf', random_state=42)),
              ('Random Forest', RandomForestClassifier(random_state=42)),
              ('AdaBoost', AdaBoostClassifier(base_estimator=RandomForestClassifier(random_state=42, max_depth=3), random_state=42)),
              ('XGBoost', XGBClassifier(base_estimator=RandomForestClassifier(random_state=42), random_state=42)),
            #   ('KNN', KNeighborsClassifier()),
]
param_grid_svc_linear = {'C': [1, 10, 50, 100, 500],
            }

param_grid_svc_poly2 = {'C': [1, 10, 50, 100, 500],
              'gamma': ['scale', 'auto', 0.1, 0.01, 0.001],
            }

param_grid_svc_rbf = {'C': [1, 10, 50, 100, 500],
              'gamma': ['scale', 'auto', 0.1, 0.01, 0.001],
            }

param_grid_rdn_forest = {'n_estimators': [50, 100, 200],
              'criterion': ['entropy'],
              'max_features': [2, 3, 5, 7, 9, 'sqrt'],
              'max_depth': [1, 3, 5, 7, 7, 8, 9, 10, 'sqrt'],
            }

param_grid_ada_boost = {'n_estimators': [100, 200],
              'learning_rate': [0.001, 0.005, 0.01, 0.1],
            }

param_grid_xg_boost = {'n_estimators': [100, 300, 500, 1000, 2000],
              'learning_rate': [0.001, 0.005, 0.01, 0.1],
            }

param_grid_knn = {'n_neighbors': [2, 3, 4, 5, 6, 7],
              'weights': ['uniform', 'distance'],
            }


params = [param_grid_svc_linear,
          param_grid_svc_poly2,
          param_grid_svc_rbf,
          param_grid_rdn_forest,
          param_grid_ada_boost,
          param_grid_xg_boost,
        #   param_grid_knn,
]

In [25]:
# Viết function tuning hyper-parameters models bằng Grid
def rename_keys(dic, pattern):
    return dict([(keys.split(pattern)[1], values) for keys, values in dic.items()])

def process_tunning_model(model_tunning_list, params, X, y, scoring='accuracy', cv_k=5):
    from sklearn.model_selection import GridSearchCV, KFold
    import time
    models_final = []
    kfold = KFold(n_splits=cv_k, shuffle=True, random_state=42)

    for model_list, param in zip(model_tunning_list, params):
        tic = time.time()
        model_name, model = model_list
        search = GridSearchCV(estimator=model, param_grid=param, cv=kfold, scoring=scoring)
        search.fit(X, y)
        model_tunned = model.set_params(**search.best_params_)
        models_final.append((model_name, model_tunned))
        toc = time.time()
        print('Model {} have been tunned in {:,.2f} seconds'.format(model_name, (toc-tic)))
    return models_final

In [26]:
X_train, X_test, y_train, y_test = data_set_2_clf
models_final = process_tunning_model(model_tunning_list, params, X_train, y_train)
models_final

Model SVC - linear have been tunned in 23.21 seconds
Model SVC - poly have been tunned in 3.61 seconds
Model SVC - rbf have been tunned in 4.91 seconds
Model Random Forest have been tunned in 148.03 seconds
Model AdaBoost have been tunned in 1,020.74 seconds
Model XGBoost have been tunned in 219.22 seconds


[('SVC - linear',
  SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
      decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
      max_iter=-1, probability=False, random_state=42, shrinking=True, tol=0.001,
      verbose=False)),
 ('SVC - poly',
  SVC(C=100, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
      decision_function_shape='ovr', degree=2, gamma='auto', kernel='poly',
      max_iter=-1, probability=False, random_state=42, shrinking=True, tol=0.001,
      verbose=False)),
 ('SVC - rbf',
  SVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
      decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
      max_iter=-1, probability=False, random_state=42, shrinking=True, tol=0.001,
      verbose=False)),
 ('Random Forest',
  RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                         criterion='entropy', max_depth=9, max_features=9,
         

In [27]:
result_model_clf_cv(models_final, data_set_2_clf)

,model,train_cv_accuracy,test_cv_accuracy,train_cv_f1,test_cv_f1,fit_time,score_time,train_accuracy,test_accuracy
2,SVC - rbf,0.840739,0.722156,0.840739,0.722156,0.028397,0.007118,0.835539,0.779661
4,AdaBoost,0.920609,0.720252,0.920609,0.720252,31.076699,1.828822,0.903592,0.785311
5,XGBoost,0.968336,0.710818,0.968336,0.710818,0.846351,0.008670,0.954631,0.706215
0,SVC - linear,0.807187,0.707080,0.807187,0.707080,0.032929,0.005440,0.786389,0.745763
3,Random Forest,0.909263,0.705121,0.909263,0.705121,0.184778,0.013365,0.892250,0.757062
1,SVC - poly,0.911625,0.697592,0.911625,0.697592,0.023028,0.006034,0.903592,0.717514


- Kết quả có được cải thiện so với model ban đầu.
- Sử dụng AdaBoost cải thiện được Random Forest.
- Thử sử dụng Voting Classifer xem thử kết quả tốt nhất có thể đạt được là bao nhiêu


In [33]:
from sklearn.ensemble import VotingClassifier
voting_model = VotingClassifier(estimators=models_final)
X_train, X_test, y_train, y_test = data_set_2_clf
voting_model.fit(X_train, y_train)
print('Accuracy Train: {:.4f}'.format(voting_model.score(X_train, y_train)))
print('Accuracy Test: {:.4f}'.format(voting_model.score(X_test, y_test)))

Accuracy Train: 0.8998
Accuracy Test: 0.7514


Evaluate model dựa trên cross-validation 5-folds

In [35]:
X_full = pd.concat([X_train, X_test], axis=0)
y_full = pd.concat([y_train, y_test], axis=0)

from sklearn.model_selection import cross_validate
cv_result = cross_validate(voting_model, X_full, y_full, cv=5, return_train_score=True, scoring=['accuracy'])
print('Accuracy Train - CV: {:.4f}'.format(cv_result['train_accuracy'].mean()))
print('Accuracy Test - CV : {:.4f}'.format(cv_result['test_accuracy'].mean()))

Accuracy Train - CV: 0.8920
Accuracy Test - CV : 0.7564


**Kết luận:**
- Kết quả cho thấy model khá ổn định khi kết quả CV cũng gần với kết quả trên test set.
- Vậy khi chuyển bài toán từ Regression sang Classification, model có thể dự đoán chính xác được đến 75% khi phân loại thành 4 nhóm.